In [31]:
# Dowload de bibliotecas
!pip -q install gdown imbalanced-learn fastapi uvicorn nest-asyncio

In [95]:

# Instalação de bibliotecas
import importlib
import os
import zipfile
import gdown
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Import do script com as funções da equipe
import script_v3 as scr
importlib.reload(scr)

<module 'script_v3' from 'e:\\New folder\\Codigos_Programacao\\Alura\\hackathon_one_flightOnTime\\hackathon_one_flightOnTime\\notebooks\\semana03\\script_v3.py'>

In [72]:
# Carregar Dados
file_id = "1207psedBKvnS0pJkDITroSzPiWrcz0ag"

# Nome do arquivo que será baixado no Colab
zip_path = "dados_vra.zip"

# Se ainda não existir o zip, baixa do Drive
if not os.path.exists(zip_path):
    url = f"https://drive.google.com/uc?id={file_id}"
    print("Baixando arquivo do Drive...")
    gdown.download(url, zip_path, quiet=False)
else:
    print("Arquivo ZIP já existe, download interrompido.")

# Extrair o conteúdo do zip
extract_folder = "dados_vra"

if not os.path.exists(extract_folder):
    print("Extraindo arquivos do ZIP...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_folder)
else:
    print("Pasta de dados já existe.")

Arquivo ZIP já existe, download interrompido.
Pasta de dados já existe.


In [73]:
pasta = "notebooks/semana03/dados_vra"

print("CWD:", os.getcwd())
print("Existe?", os.path.exists(pasta))
print("Absoluto:", os.path.abspath(pasta))

CWD: e:\New folder\Codigos_Programacao\Alura\hackathon_one_flightOnTime\hackathon_one_flightOnTime\notebooks\semana03
Existe? False
Absoluto: e:\New folder\Codigos_Programacao\Alura\hackathon_one_flightOnTime\hackathon_one_flightOnTime\notebooks\semana03\notebooks\semana03\dados_vra


In [61]:
# Carregamento do Dataset
df = scr.carregar_dataset_base(
    pasta= "dados_vra/dados_vra",  # ajuste aqui se necessário
    sep=";",
    encoding="latin-1",
    skiprows=1,
    renomear=True,
    converter_datas=True,
)

df.head()

e:\New folder\Codigos_Programacao\Alura\hackathon_one_flightOnTime\hackathon_one_flightOnTime\notebooks\semana03\script_v3.py:160: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


,empresa_aerea,numero_voo,codigo_autorizacao_di,codigo_tipo_linha,aerodromo_origem,aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,codigo_justificativa
0,TAM,3447,0,N,SBGR,SBFI,2021-01-09 20:25:00,2021-01-09 20:21:00,2021-01-09 22:05:00,2021-01-09 21:56:00,REALIZADO,NaN
1,TAM,3447,0,N,SBGR,SBFI,2021-01-10 20:25:00,2021-01-10 20:24:00,2021-01-10 22:05:00,2021-01-10 21:55:00,REALIZADO,NaN
2,TAM,3447,0,N,SBGR,SBFI,2021-01-11 20:25:00,2021-01-11 20:27:00,2021-01-11 22:05:00,2021-01-11 21:58:00,REALIZADO,NaN
3,TAM,3447,0,N,SBGR,SBFI,2021-01-12 20:25:00,2021-01-12 20:24:00,2021-01-12 22:05:00,2021-01-12 21:58:00,REALIZADO,NaN
4,TAM,3447,0,N,SBGR,SBFI,2021-01-13 20:25:00,2021-01-13 20:16:00,2021-01-13 22:05:00,2021-01-13 21:48:00,REALIZADO,NaN


In [62]:
df_flags = scr.criar_flags_qualidade_basicas(df)

df_flags[[
    "flag_partida_prevista_ausente",
    "flag_partida_real_ausente",
    "flag_data_fora_periodo"
]].mean()

flag_partida_prevista_ausente    0.041119
flag_partida_real_ausente        0.042030
flag_data_fora_periodo           0.000000
dtype: float64

In [63]:
df_model = scr.criar_target_atrasado(df_flags, limite_min=15)

df_model[scr.TARGET_COL].value_counts(normalize=True)
df_model.shape

(3644100, 17)

In [64]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3644100 entries, 0 to 3968417
Data columns (total 17 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   empresa_aerea                  object        
 1   numero_voo                     object        
 2   codigo_autorizacao_di          object        
 3   codigo_tipo_linha              object        
 4   aerodromo_origem               object        
 5   aerodromo_destino              object        
 6   partida_prevista               datetime64[ns]
 7   partida_real                   datetime64[ns]
 8   chegada_prevista               datetime64[ns]
 9   chegada_real                   datetime64[ns]
 10  situacao_voo                   object        
 11  codigo_justificativa           float64       
 12  flag_partida_prevista_ausente  bool          
 13  flag_partida_real_ausente      bool          
 14  flag_data_fora_periodo         bool          
 15  atraso_partida_min  

# Split estratificado

In [65]:
df_train, df_test = scr.criar_split_estratificado(df_model, coluna_target=scr.TARGET_COL)
print('Formato dos dados de treino: ', df_train.shape)
print('Formato dos dados de teste: ', df_test.shape)

Formato dos dados de treino:  (2915280, 17)
Formato dos dados de teste:  (728820, 17)


In [66]:
# Definição das colunas que irão para o modelo
cfg = scr.FeatureConfig(
    numeric_features=[
        # criadas pelo função DatasFeaturesTransformer
        "hora_dia",
        "dia_semana",
        "mes_ano",
        "fim_de_semana",
        "alta_temporada",

        # criadas pelo pela função MediaAtrasoTransformer
        "media_atraso_empresa",
        "media_atraso_origem",
        "media_atraso_destino",
    ],
    categorical_features=[
        # originais
        "empresa_aerea",
        "codigo_tipo_linha",
        "aerodromo_origem",
        "aerodromo_destino",
        "situacao_voo",

        # criada pela função DatasFeaturesTransformer
        "periodo_dia",
    ],
)

In [67]:
# Treinamento do Modelo com o split estratificado
model = LogisticRegression(max_iter=200)

out = scr.treinar_classificador(
    df_train=df_train,
    df_test=df_test,
    cfg=cfg,
    model=model,
    target=scr.TARGET_COL,
    use_smote=False
)

pipe = out["pipeline"]
metrics = out["metrics"]

metrics["confusion_matrix"], metrics.get("roc_auc")

([[607446, 4818], [109491, 7065]], 0.6596363059445411)

# Usando o split_Temporal

In [98]:
df_train, df_val, df_test = scr.criar_split_temporal_train_val_test(
    df=df_model,
    time_col=scr.COL_DT_BASE,
)

print('Formato dos dados de treino: ', df_train.shape)
print('Formato dos dados de validação: ', df_val.shape)
print('Formato dos dados de teste: ', df_test.shape)

Formato dos dados de treino:  (2550870, 17)
Formato dos dados de validação:  (364409, 17)
Formato dos dados de teste:  (728821, 17)


In [85]:
df_train['partida_prevista'].dt.year.value_counts().sort_index()

partida_prevista
2021    584427
2022    810428
2023    894067
2024    261948
Name: count, dtype: int64

In [81]:
print(df_train['partida_prevista'].min(), df_train['partida_prevista'].max())
print(df_test['partida_prevista'].min(), df_test['partida_prevista'].max())


2021-01-01 00:05:00 2024-04-16 10:20:00
2024-09-09 08:00:00 2025-07-01 07:10:00


## Usando proporçóes

In [99]:
# Treinamento do Modelo com o split estratificado
model = LogisticRegression(max_iter=200)

out_val = scr.treinar_classificador(
    df_train=df_train,
    df_test=df_val,
    cfg=cfg,
    model=model,
    target=scr.TARGET_COL,
    use_smote=False
)

pipe = out_val["pipeline"]
metrics = out_val["metrics"]

metrics["confusion_matrix"], metrics.get("roc_auc")

([[307625, 2974], [49611, 4199]], 0.6388094077905119)

Train e Test

In [100]:
# Treinamento do Modelo com o split estratificado
model = LogisticRegression(max_iter=200)

out_val = scr.treinar_classificador(
    df_train=df_train,
    df_test=df_test,
    cfg=cfg,
    model=model,
    target=scr.TARGET_COL,
    use_smote=False
)

pipe = out_val["pipeline"]
metrics = out_val["metrics"]

metrics["confusion_matrix"], metrics.get("roc_auc")

([[595452, 4054], [124424, 4891]], 0.6364117487096171)

Train+val e test

In [101]:
df_train_full = pd.concat([df_train, df_val],axis=0)

In [102]:
# Treinamento do Modelo com o split estratificado
model = LogisticRegression(max_iter=200)

out_val = scr.treinar_classificador(
    df_train=df_train_full,
    df_test=df_test,
    cfg=cfg,
    model=model,
    target=scr.TARGET_COL,
    use_smote=False
)

pipe = out_val["pipeline"]
metrics = out_val["metrics"]

metrics["confusion_matrix"], metrics.get("roc_auc")

([[595947, 3559], [124474, 4841]], 0.6420109748832079)

Problema

partida_prevista

| Ano  | Número de voos |
|------|----------------|
| 2021 | 584,427        |
| 2022 | 810,428        |
| 2023 | 894,067        |
| 2024 | 261,948        |


- Atualmente, todos os anos estão no treino, então o modelo já “viu” padrões de 2023-2024 ao treinar, e qualquer métrica calculada sobre esses anos não representa uso real futuro

- Os 70% iniciais de linhas correspondem a datas de todos os anos, porque os dados não estão uniformemente distribuídos por ano.

- Como há muito mais voos de 2023, 2022 do que de 2021, a divisão por fração cria treino que contém até os anos mais recentes.

**Conclusão**: o split fracional não garante validade temporal quando a frequência de registros por ano é desigual.



## Uso de filtros para separação dos dados por anos

**Estrátegia**:

- Ajustar a função criar_split_temporal_train_val_test para usar anos específicos, em vez de proporções.

- Recontar quantos voos estão em cada split para ter certeza que:
    - **Treino** = anos mais antigos
    - **Validação** = anos intermediários
    - **Teste** = anos mais recentes

In [92]:
df_train, df_val, df_test = scr.criar_split_temporal_train_val_test_mod(
    df=df_model,
    time_col=scr.COL_DT_BASE,
)

print(f"Split realizado:")
print(f"  Treino: {df_train.shape[0]} linhas ({df_train[scr.COL_DT_BASE].min().date()}  {df_train[scr.COL_DT_BASE].max().date()})")
print(f"  Validação: {df_val.shape[0]} linhas ({df_val[scr.COL_DT_BASE].min().date()}  {df_val[scr.COL_DT_BASE].max().date()})")
print(f"  Teste: {df_test.shape[0]} linhas ({df_test[scr.COL_DT_BASE].min().date()}  {df_test[scr.COL_DT_BASE].max().date()})")

Split realizado:
  Treino: 1393022 linhas (2021-01-01  2022-12-31)
  Validação: 893967 linhas (2022-12-31  2023-12-30)
  Teste: 1357111 linhas (2023-12-31  2025-07-01)


In [93]:
# Treinamento do Modelo com o split estratificado
model = LogisticRegression(max_iter=200)

out_val = scr.treinar_classificador(
    df_train=df_train,
    df_test=df_val,
    cfg=cfg,
    model=model,
    target=scr.TARGET_COL,
    use_smote=False
)

pipe = out_val["pipeline"]
metrics = out_val["metrics"]

metrics["confusion_matrix"], metrics.get("roc_auc")

([[723531, 7543], [154785, 8108]], 0.6419276567851235)

In [94]:
# Treinamento do Modelo com o split estratificado
model = LogisticRegression(max_iter=200)

out_val = scr.treinar_classificador(
    df_train=df_train,
    df_test=df_test,
    cfg=cfg,
    model=model,
    target=scr.TARGET_COL,
    use_smote=False
)

pipe = out_val["pipeline"]
metrics = out_val["metrics"]

metrics["confusion_matrix"], metrics.get("roc_auc")

([[1115783, 9772], [222379, 9177]], 0.6166163960604897)

In [96]:
df_train_full = pd.concat([df_train, df_val], axis=0)

In [97]:
# Treinamento do Modelo com o split estratificado
model = LogisticRegression(max_iter=200)

out_val = scr.treinar_classificador(
    df_train=df_train_full,
    df_test=df_test,
    cfg=cfg,
    model=model,
    target=scr.TARGET_COL,
    use_smote=False
)

pipe = out_val["pipeline"]
metrics = out_val["metrics"]

metrics["confusion_matrix"], metrics.get("roc_auc")

([[1118003, 7552], [222123, 9433]], 0.6262892049694876)

## Recomendações

**Tabela do resumo total**

| Tipo de Split                         | Conjunto               | Confusion Matrix                  | ROC-AUC        |
|--------------------------------------|-----------------------|----------------------------------|----------------|
| Estratificado                         | Treino vs Teste       | [[607446, 4818], [109491, 7065]] | 0.6596         |
| Temporal (proporção)                  | Treino vs Validação   | [[307625, 2974], [49611, 4199]]  | 0.6388         |
| Temporal (proporção)                  | Treino vs Teste       | [[595452, 4054], [124424, 4891]] | 0.6364         |
| Temporal (proporção)                  | Treino+Val vs Teste   | [[595947, 3559], [124474, 4841]] | 0.6420         |
| Temporal (filtro anos específicos)   | Treino vs Validação   | [[723531, 7543], [154785, 8108]] | 0.6419         |
| Temporal (filtro anos específicos)   | Treino vs Teste       | [[1115783, 9772], [222379, 9177]]| 0.6166         |
| Temporal (filtro anos específicos)   | Treino+Val vs Teste   | [[1118003, 7552], [222123, 9433]]| 0.6263         |

# Comparação de Estratégias de Split e Recomendações

A validação do modelo foi realizada usando diferentes estratégias de divisão dos dados:

| Tipo de Split                         | Conjunto               | Confusion Matrix                  | ROC-AUC        |
|--------------------------------------|-----------------------|----------------------------------|----------------|
| Estratificado                         | Treino vs Teste       | [[607446, 4818], [109491, 7065]] | 0.6596         |
| Temporal (proporção)                  | Treino vs Validação   | [[307625, 2974], [49611, 4199]]  | 0.6388         |
| Temporal (proporção)                  | Treino vs Teste       | [[595452, 4054], [124424, 4891]] | 0.6364         |
| Temporal (proporção)                  | Treino+Val vs Teste   | [[595947, 3559], [124474, 4841]] | 0.6420         |
| Temporal (filtro anos específicos)   | Treino vs Validação   | [[723531, 7543], [154785, 8108]] | 0.6419         |
| Temporal (filtro anos específicos)   | Treino vs Teste       | [[1115783, 9772], [222379, 9177]]| 0.6166         |
| Temporal (filtro anos específicos)   | Treino+Val vs Teste   | [[1118003, 7552], [222123, 9433]]| 0.6263         |

## Observações

1. **Split Estratificado**
   - ROC-AUC mais alto (0.6596), mas não representa uso real.
   - O modelo já “viu” dados recentes durante o treino → métricas otimistas.

2. **Split Temporal (proporção 70/10/20)**
   - ROC-AUC ligeiramente menor (0.636-0.642).
   - Respeita a ordem temporal, mas frações podem misturar anos, não simulando exatamente produção futura.

3. **Split Temporal com cutoff por anos**
   - ROC-AUC mais baixo (0.616–0.642), especialmente no teste.
   - Avalia o modelo com dados futuros não vistos → representa **cenário de produção real**.
   - Mostra que o modelo aprende padrões passados, mas performance futura é naturalmente menor.

## Recomendações

- Para **validação realista** do modelo, priorizar **split temporal com cutoff de anos específicos**.
- Splits estratificados devem ser usados apenas para **teste rápido**, mas não refletem o desempenho real em produção.
- Métricas mais baixas nos splits temporais indicam que o modelo precisa ser interpretado considerando **desempenho futuro**, não apenas resultados históricos.
- Isso quer dizer que o resultado encontrado através do split temporal reflete desempenho futuro real: o quanto o modelo conseguirá prever atrasos de voos que ele nunca viu antes.
